<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2023-09-20 20:28:22--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2023-09-20 20:28:22--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2023-09-20 20:28:23--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [4]:
# Posibles bandas
os.listdir("./songs_dataset/")

['Kanye_West.txt',
 'bob-marley.txt',
 'bruce-springsteen.txt',
 'lorde.txt',
 'radiohead.txt',
 'nirvana.txt',
 'al-green.txt',
 'beatles.txt',
 'alicia-keys.txt',
 'adele.txt',
 'joni-mitchell.txt',
 'drake.txt',
 'notorious-big.txt',
 'notorious_big.txt',
 'michael-jackson.txt',
 'r-kelly.txt',
 'jimi-hendrix.txt',
 'bieber.txt',
 'Lil_Wayne.txt',
 'ludacris.txt',
 'dr-seuss.txt',
 'kanye-west.txt',
 'nursery_rhymes.txt',
 'eminem.txt',
 'lin-manuel-miranda.txt',
 'bob-dylan.txt',
 'janisjoplin.txt',
 'nickelback.txt',
 'bjork.txt',
 'missy-elliott.txt',
 'nicki-minaj.txt',
 'paul-simon.txt',
 'kanye.txt',
 'blink-182.txt',
 'dj-khaled.txt',
 'britney-spears.txt',
 'bruno-mars.txt',
 'rihanna.txt',
 'johnny-cash.txt',
 'lady-gaga.txt',
 'lil-wayne.txt',
 'cake.txt',
 'prince.txt',
 'amy-winehouse.txt',
 'patti-smith.txt',
 'disney.txt',
 'leonard-cohen.txt',
 'dickinson.txt',
 'dolly-parton.txt']

In [5]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)
df.head()

<ipython-input-5-89d9bc44a57b>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)


,0
0,"""Don't worry about a thing,"
1,'Cause every little thing gonna be all right.
2,"Singin': ""Don't worry about a thing,"
3,"'Cause every little thing gonna be all right!""..."
4,"Smiled with the risin' sun,"


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2218


### 1 - Preprocesamiento

In [7]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [8]:
# Demos un vistazo
sentence_tokens[:2]

[["don't", 'worry', 'about', 'a', 'thing'],
 ["'cause", 'every', 'little', 'thing', 'gonna', 'be', 'all', 'right']]

### 2 - Crear los vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [10]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [11]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [12]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2218


In [13]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 532


### 3 - Entrenar el modelo generador

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 150638.921875
Loss after epoch 1: 93187.125
Loss after epoch 2: 92462.078125
Loss after epoch 3: 92596.03125
Loss after epoch 4: 92077.78125
Loss after epoch 5: 88974.1875
Loss after epoch 6: 82641.75
Loss after epoch 7: 79397.0
Loss after epoch 8: 75730.625
Loss after epoch 9: 72150.9375
Loss after epoch 10: 70748.375
Loss after epoch 11: 68126.9375
Loss after epoch 12: 62096.0
Loss after epoch 13: 60407.375
Loss after epoch 14: 60351.625
Loss after epoch 15: 60054.5
Loss after epoch 16: 58789.375
Loss after epoch 17: 58724.0
Loss after epoch 18: 58439.25
Loss after epoch 19: 57414.25


(218093, 359260)

### 4 - Ensayar

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('given', 0.8143353462219238),
 ('protect', 0.8042460083961487),
 ("you've", 0.8037887811660767),
 ('nobody', 0.8014757633209229),
 ("feelin'", 0.7951468825340271),
 ('really', 0.7939077615737915),
 ('ask', 0.7758776545524597),
 ('message', 0.7723156809806824),
 ('mean', 0.7716444134712219),
 ('survive', 0.7477133870124817)]

In [23]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('walk', -0.19966642558574677),
 ('ganja', -0.20876391232013702),
 ('paradise', -0.2094326764345169),
 ("pimper's", -0.21143466234207153),
 ('a', -0.22156809270381927),
 ("that's", -0.22306372225284576),
 ("'ypocrites", -0.22687889635562897),
 ("'pon", -0.2317732721567154),
 ("smoke'a", -0.2357722669839859),
 ('oo', -0.23827940225601196)]

Con la palabra "love", lo que sucede es que las mayores coincidencias son con palabras que se nombran en frases, o se utilizan en oraciones.
Sucede lo contrario con las que menos se relacionan, son palabras que no tienen sentido utilizarlas junto a "love" o no se utilizan en una misma oración

In [25]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["every"], topn=10)

[('night', 0.8873708844184875),
 ('three', 0.8686659336090088),
 ('birds', 0.8509165048599243),
 ("'cause", 0.8427338004112244),
 ('day', 0.8267289400100708),
 ('word', 0.8088815212249756),
 ('little', 0.7607012391090393),
 ('new', 0.7486817836761475),
 ('same', 0.7068617343902588),
 ('only', 0.7063501477241516)]

In [29]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["woman"], topn=5)

[('cry', 0.9774102568626404),
 ('fowl', 0.9347646832466125),
 ('no', 0.9244766235351562),
 ('tears', 0.916095495223999),
 ('call', 0.8935620784759521)]

En este caso, se puede ver que la mayor coincidencia se da con la palabra "cry", esto se debe a una de sus canciones más conocidas llamada "No Woman No Cry".

In [31]:
w2v_model.wv.most_similar(positive=["three"])

[('birds', 0.9717826247215271),
 ('word', 0.8823035359382629),
 ('every', 0.868665874004364),
 ('song', 0.8633241057395935),
 ('two', 0.8630540370941162),
 ('little', 0.8501969575881958),
 ('night', 0.8392566442489624),
 ('only', 0.8381635546684265),
 ('natural', 0.8369463682174683),
 ('mystic', 0.8355417251586914)]

Sucede lo mismo en este caso, la palabra que más se relaciona es una palabra que está asociada a una frase que se repite mucho en una canción.

In [39]:
w2v_model.wv.most_similar(positive=["jamaica"])

[("y'all", 0.9675228595733643),
 ('smile', 0.9351271986961365),
 ('wise', 0.8689877986907959),
 ('yard', 0.8096752166748047),
 ('together', 0.7774273157119751),
 ('get', 0.7739351987838745),
 ('top', 0.7701151967048645),
 ("they're", 0.7658965587615967),
 ('name', 0.7630153894424438),
 ("let's", 0.7545446753501892)]

En este caso se puede ver que la palabra Jamaica tiene mucha relación con palabras de felicidad como "smile", o palabras como "they're" que es una palabra que se puede utilizar con Jamaica.

### 5 - Visualizar agrupación de vectores

In [32]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model):
    num_dimensions = 2

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [33]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.